In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [3]:
g = dataset[0]

In [4]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [5]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [6]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.132 (best 0.132), test acc: 0.117 (best 0.117)
In epoch 5, loss: 1.897, val acc: 0.524 (best 0.524), test acc: 0.530 (best 0.530)
In epoch 10, loss: 1.815, val acc: 0.638 (best 0.646), test acc: 0.649 (best 0.657)
In epoch 15, loss: 1.707, val acc: 0.666 (best 0.666), test acc: 0.685 (best 0.685)
In epoch 20, loss: 1.574, val acc: 0.696 (best 0.696), test acc: 0.719 (best 0.719)
In epoch 25, loss: 1.419, val acc: 0.714 (best 0.714), test acc: 0.731 (best 0.731)
In epoch 30, loss: 1.247, val acc: 0.738 (best 0.738), test acc: 0.744 (best 0.741)
In epoch 35, loss: 1.069, val acc: 0.750 (best 0.750), test acc: 0.759 (best 0.756)
In epoch 40, loss: 0.895, val acc: 0.756 (best 0.756), test acc: 0.762 (best 0.762)
In epoch 45, loss: 0.733, val acc: 0.762 (best 0.762), test acc: 0.770 (best 0.768)
In epoch 50, loss: 0.592, val acc: 0.766 (best 0.766), test acc: 0.774 (best 0.774)
In epoch 55, loss: 0.474, val acc: 0.768 (best 0.768), test acc: 0.781 (best 0

In [8]:
g = g.to('cpu')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cpu')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.160 (best 0.160), test acc: 0.174 (best 0.174)
In epoch 5, loss: 1.890, val acc: 0.544 (best 0.544), test acc: 0.594 (best 0.594)
In epoch 10, loss: 1.803, val acc: 0.640 (best 0.640), test acc: 0.670 (best 0.670)
In epoch 15, loss: 1.692, val acc: 0.698 (best 0.698), test acc: 0.714 (best 0.714)
In epoch 20, loss: 1.558, val acc: 0.716 (best 0.716), test acc: 0.729 (best 0.729)
In epoch 25, loss: 1.402, val acc: 0.742 (best 0.742), test acc: 0.736 (best 0.736)
In epoch 30, loss: 1.231, val acc: 0.748 (best 0.748), test acc: 0.749 (best 0.746)
In epoch 35, loss: 1.054, val acc: 0.742 (best 0.748), test acc: 0.757 (best 0.746)
In epoch 40, loss: 0.881, val acc: 0.760 (best 0.760), test acc: 0.760 (best 0.755)
In epoch 45, loss: 0.722, val acc: 0.766 (best 0.766), test acc: 0.763 (best 0.761)
In epoch 50, loss: 0.584, val acc: 0.776 (best 0.776), test acc: 0.772 (best 0.771)
In epoch 55, loss: 0.468, val acc: 0.776 (best 0.778), test acc: 0.783 (best 0